In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Load your product data
products_df = pd.read_csv('DummyMyntraDataset.csv')

# Example column names, adjust as per your dataset
product_columns = ['p_id', 'name', 'price', 'colour', 'brand', 'img', 'ratingCount', 'avg_rating',
                   'Body Shape ID', 'Body or Garment Size', 'Bottom Closure', 'Bottom Fabric',
                   'Bottom Pattern', 'Bottom Type', 'Dupatta', 'Dupatta Border', 'Dupatta Fabric',
                   'Dupatta Pattern', 'Main Trend', 'Neck', 'Number of Pockets', 'Occasion',
                   'Pattern Coverage', 'Print or Pattern Type', 'Sleeve Length', 'Sleeve Styling',
                   'Slit Detail', 'Stitch', 'Sustainable', 'Top Design Styling', 'Top Fabric',
                   'Top Hemline', 'Top Length', 'Top Pattern', 'Top Shape', 'Top Type', 'Waistband',
                   'Wash Care', 'Weave Pattern', 'Weave Type', 'Ornamentation']


In [2]:
# Combine relevant product attributes into a single string for TF-IDF
products_df['combined_features'] = products_df[[
    'colour', 'brand', 'Bottom Pattern', 'Bottom Type', 'Dupatta', 'Dupatta Border', 'Dupatta Fabric',
    'Dupatta Pattern', 'Main Trend', 'Neck', 'Occasion', 'Print or Pattern Type', 'Sleeve Length',
    'Sleeve Styling', 'Stitch', 'Top Design Styling', 'Top Fabric', 'Top Hemline', 'Top Pattern',
    'Top Shape', 'Top Type', 'Waistband', 'Wash Care', 'Weave Pattern', 'Weave Type', 'Ornamentation'
]].fillna('').agg(' '.join, axis=1)

# Apply TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=500)  # Adjust max_features as needed
tfidf_matrix = vectorizer.fit_transform(products_df['combined_features'])


In [4]:
from sklearn.neighbors import NearestNeighbors

# Instantiate the Nearest Neighbors model
content_model = NearestNeighbors(metric='cosine', algorithm='brute')

# Fit the model with your TF-IDF matrix
content_model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [5]:
def get_content_recommendations(product_id, model, n_recommendations=5):
    try:
        product_index = products_df[products_df['p_id'] == product_id].index[0]
        distances, indices = model.kneighbors(tfidf_matrix[product_index], n_neighbors=n_recommendations + 1)
        recommended_product_indices = indices.flatten()[1:]  # Exclude the product itself
        return products_df.iloc[recommended_product_indices]
    except IndexError:
        print(f"Product ID {product_id} not found in the dataset.")
        return pd.DataFrame()

# Example of getting content-based recommendations
product_id = products_df['p_id'].iloc[0]  # Example product_id
recommendations = get_content_recommendations(product_id, content_model)
print(recommendations)


        p_id                                               name  price colour  \
79  17447636  Khushal K Women Green Ethnic Motifs Printed Go...   5699  Green   
18  14649544  JAIPURI BUNAAI Women Blue & Orange Ethnic Moti...   3999   Blue   
9   17048604  Khushal K Women Green & Pink Printed Pure Cott...   5299  Green   
20  13810898  Indo Era Women Blue & Green Printed Kurta with...   4599   Blue   
32  16331336  Anubhutee Women Teal Ethnic Motifs Printed Emp...   5299   Teal   

             brand                                                img  \
79       Khushal K  http://assets.myntassets.com/assets/images/174...   
18  JAIPURI BUNAAI  http://assets.myntassets.com/assets/images/pro...   
9        Khushal K  http://assets.myntassets.com/assets/images/170...   
20        Indo Era  http://assets.myntassets.com/assets/images/138...   
32       Anubhutee  http://assets.myntassets.com/assets/images/163...   

    ratingCount  avg_rating      Body Shape ID     Body or Garment Size  .

In [8]:

def get_product_image(num_pockets, color, brand):
    try:
        # Filter the dataset based on user inputs
        filtered_products = products_df[
            (products_df['Number of Pockets'] == num_pockets) &
            (products_df['colour'] == color) &
            (products_df['brand'] == brand)
        ]
        
        if not filtered_products.empty:
            # Assuming there's only one matching product, take the first one
            img_url = filtered_products['img'].iloc[0]
            return img_url
        else:
            print("No product found with the given criteria.")
            return None
    except KeyError:
        print("Invalid column name specified.")
        return None

# Example usage
num_pockets = 1  # Example user input
color = 'Yellow'   # Example user input
brand = 'Biba'   # Example user input

image_url = get_product_image(num_pockets, color, brand)
if image_url:
    print(f"Image URL: {image_url}")
    # Now you can use this image URL to display or process the image
else:
    print("No image found.")


Image URL: http://assets.myntassets.com/assets/images/14848486/2021/9/1/849c44d7-0274-49fa-bb15-c5d786b74e351630497951879-Biba-Women-Yellow-Printed-Regular-Gotta-Patti-Kurta-with-Tro-1.jpg


In [ ]:
import pandas as pd

# Example dataset (replace with your actual dataset)
products_df = pd.DataFrame({
    'p_id': [1, 2, 3, 4],
    'name': ['Product A', 'Product B', 'Product C', 'Product D'],
    'price': [100, 120, 90, 110],
    'colour': ['Blue', 'Red', 'Black', 'Blue'],
    'brand': ['Brand X', 'Brand Y', 'Brand Z', 'Brand X'],
    'img': ['image_url_1.jpg', 'image_url_2.jpg', 'image_url_3.jpg', 'image_url_4.jpg'],
    'ratingCount': [50, 40, 30, 45],
    'avg_rating': [4.5, 4.0, 3.5, 4.2],
    'Number of Pockets': [4, 2, 3, 4]
})

def get_product_images(num_pockets, color, brand, n_recommendations=3):
    try:
        # Filter the dataset based on user inputs
        filtered_products = products_df[
            (products_df['Number of Pockets'] == num_pockets) &
            (products_df['colour'] == color) &
            (products_df['brand'] == brand)
        ]
        
        if not filtered_products.empty:
            # Take up to n_recommendations products
            recommended_images = filtered_products.head(n_recommendations)['img'].tolist()
            return recommended_images
        else:
            print("No products found with the given criteria.")
            return []
    except KeyError:
        print("Invalid column name specified.")
        return []

# Example usage
num_pockets = 4  # Example user input
color = 'Blue'   # Example user input
brand = 'Brand X'   # Example user input

recommendations = get_product_images(num_pockets, color, brand, n_recommendations=3)
if recommendations:
    print("Recommended Images:")
    for img_url in recommendations:
        print(img_url)
else:
    print("No images found.")
